In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [ ]:
######### mediapipe Holistic model ########### 

In [ ]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [ ]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [ ]:
def draw_styled_landmarks(image, results):
    #Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [ ]:
######## Loading Photos ######## 
Dataset_Orign = 'data'  
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    # Iterate through all folders in the Dataset_Orign
    for folder in os.listdir(Dataset_Orign):
        # Get the full path of the folder
        folder_path = os.path.join(Dataset_Orign, folder)
        
        # Check if it's a directory
        if os.path.isdir(folder_path):
            # Iterate through all photos in the directory
            for Photo in os.listdir(folder_path):
                # Get the full path of the photo_file
                photo_path = os.path.join(folder_path, Photo)

                # Check if it's a file
                if os.path.isfile(photo_path):
                    # Read feed
                    frame = cv2.imread(photo_path)

                    # Check if the frame was successfully read
                    if frame is not None:
                        # Make detections
                        img, results = mediapipe_detection(frame, holistic)
                        print(results)

                        # Draw landmarks
                        draw_landmarks(img, results)

                        # Wait for a key press
                        cv2.waitKey(0)

    # Close all windows
    cv2.destroyAllWindows()

In [ ]:
len(results.pose_landmarks.landmark)

In [ ]:
results

In [ ]:
draw_landmarks(frame, results)

In [ ]:
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

In [ ]:
###### Extracting Keypoints #####

In [ ]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [ ]:
len(results.pose_landmarks.landmark)

In [ ]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [ ]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

In [ ]:
result_test = extract_keypoints(results)

In [ ]:
result_test

In [ ]:
np.save('0', result_test)
np.load('0.npy')

In [ ]:
###### Making Folders for dataset ######

In [ ]:
# Path for exported data, numpy arrays
DATASET = os.path.join('Dataset') 

# Actions that we try to detect
actions = np.array(['call', 'mute', 'peace'])

In [ ]:
for action in actions: 
      try: 
          os.makedirs(os.path.join(DATASET, action))
      except:
           pass

In [ ]:
####### Updating orign_data photos with npy extintion and loading it to dataset folder #######

In [ ]:
mp_holistic = mp.solutions.holistic

# Update the path
Dataset_Orign = r'data'

# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    # Loop through actions
    for action in actions:
        # Create action folder if it doesn't exist
        folder = os.path.join(Dataset_Orign, action)
        os.makedirs(folder, exist_ok=True)
        
        # Loop through folders in the Dataset_Orign
        for photo in os.listdir(Dataset_Orign):
            photo_path = os.path.join(Dataset_Orign, photo)
            output = os.path.join(DATASET, photo)
            # Check if it is a directory
            if os.path.isdir(photo_path):
                count = 0
                # Loop through files in the folder
                for frame_Photo in os.listdir(photo_path):
                    frame_path = os.path.join(photo_path, frame_Photo)
                    
                    # Check if the file is an image
                    if frame_Photo.endswith(('.jpg', '.jpeg', '.png')):
                        # Read image
                        frame = cv2.imread(frame_path)
                        
                        # Ensure that the frame is not None 
                        if frame is not None:
                            # Make detections using MediaPipe Holistic
                            img, results = mediapipe_detection(frame, holistic)
                            
                            # Draw landmarks
                            draw_styled_landmarks(img, results)
                            
                            # Export keypoints
                            keypoints = extract_keypoints(results)
                            
                            # Save keypoints as .npy file in the action folder
                            npy_file = f"{count}.npy"
                            npy_file = os.path.splitext(frame_Photo)[0] + '.npy'
                            npy_path = os.path.join(output, npy_file)
                            np.save(npy_path, keypoints)
                            count+=1
                        else:
                            print(f"An error occured with reading image: {frame_path}")

print("Done")

In [ ]:
cv2.destroyAllWindows()

In [ ]:
##### Creating Labels #####

In [ ]:
import sklearn
import pickle
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}

In [ ]:
label_map

In [ ]:
import re
sequences, labels = [], []
for action in actions:
    for file in os.listdir(os.path.join(DATASET, action)):
        # Use regular expression to extract numeric part of the file name
        match = re.match(r'(\d+)', file)
        if match:
            sequence_number = int(match.group(1))  # Extract the numeric part
            window = []
            res = np.load(os.path.join(DATASET, action, file))
            window.append(res)
            sequences.append(window)
            labels.append(label_map[action])

In [ ]:
sequences

In [ ]:
np.array(sequences).shape

In [ ]:
np.array(labels).shape

In [ ]:
X = np.array(sequences)

In [ ]:
X

In [ ]:
X.shape

In [ ]:
Y = to_categorical(labels).astype(int)

In [ ]:
Y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.05)

In [ ]:
X_train.shape

In [ ]:
y_test.shape

In [ ]:
##### Building And trainning LSTM nueral network ####

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
from keras.layers import Dropout, BatchNormalization

model = Sequential()
model.add(LSTM(128, return_sequences=True, activation='tanh', input_shape=(1, 258)))
model.add(Dropout(0.2))  # Add dropout regularization
model.add(BatchNormalization())  # Add batch normalization
model.add(LSTM(256, return_sequences=True, activation='tanh'))
model.add(Dropout(0.2))  # Add dropout regularization
model.add(BatchNormalization())  # Add batch normalization
model.add(LSTM(128, return_sequences=False, activation='tanh'))
model.add(Dropout(0.2))  # Add dropout regularization
model.add(BatchNormalization())  # Add batch normalization
model.add(Dense(128, activation='tanh'))
model.add(Dropout(0.2))  # Add dropout regularization
model.add(BatchNormalization())  # Add batch normalization
model.add(Dense(64, activation='tanh'))
model.add(Dropout(0.2))  # Add dropout regularization
model.add(BatchNormalization())  # Add batch normalization
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
actions.shape[0]

In [ ]:
res = [.7, 0.2, 0.1]

In [ ]:
actions[np.argmax(res)]

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

In [ ]:
model.summary()

In [ ]:
##### Making Predictions #####

In [ ]:
res = model.predict(X_test)

In [ ]:
actions[np.argmax(res[3])]

In [ ]:
actions[np.argmax(y_test[3])]

In [ ]:
##### Saving Weights #######

In [ ]:
model.save('action.h5')

In [ ]:
model.load_weights('action.h5')

In [ ]:
##### Confusion Matrix and accuracy #####

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_test)

In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
yhat

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
acc=accuracy_score(ytrue, yhat)
print("The accuracy is : ",acc)

In [ ]:
##### Testing #####

In [ ]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
     output_frame = input_frame.copy()
     for num, prob in enumerate(res):
         cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
         cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
     return output_frame

In [ ]:
plt.figure(figsize=(18,18))

In [ ]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
sequence.reverse()

In [ ]:
len(sequence)

In [ ]:
sequence.append('def')

In [ ]:
sequence.reverse()

In [ ]:
sequence[-30:]

In [ ]:
res[np.argmax(res)] > threshold

In [ ]:
model.predict(np.expand_dims(X_test[0], axis=0))